Fukushima Heatmap: Subsecting Data
===============================
In some cases, the amount of data available is too much to graph all at once. While the Fukushima set is small enough to fit comfortably in memory, we can still use it to showcase some techniques for handling much larger sets. In this case, we will still process the entire set of data, but do so in coordinate cells; each cell is averaged to create a heatmap. You can configure the number of cells created; the more there are, the less data is held in memory at a time, but the more queries are done overall.


Configuring the Graph
--------------------------

First, we configure the number of cells as described above. There's also a fair bit of configuration that goes into how the heatmap is displayed. Feel free to tweak these settings to maximize how readable the data is for you personally.

In [ ]:
# Number of cells along a side; this number squared is the total number of cells
CELLS_PER_SIDE = 12

# How the cities are marked. Font/marker color and size, label outline and size
COLOR_CITIES = 'white'
FONT_SIZE=12
COLOR_OUTLINE = 'black'
OUTLINE_SIZE = 5
AREA_CITIES = 50

# Heatmap colormap, see https://matplotlib.org/users/colormaps.html#miscellaneous
COLORMAP = "viridis"

print('Config loaded. Re-run this cell to update how the graph is displayed.')

Opening a connection and finding all dates
--------------------------------------------------

After we define our constants, we open a connection to our database of interest, and find what dates are available to us. We'll track data from each date separately.

In [ ]:
import sina.datastores.sql as sina_sql

DATABASE = '/collab/usr/gapps/wf/examples/data/fukushima/fukushima.sqlite'

# Identify the coordinates, label, and label orientation for the power
# plant and selected cities as points of reference.
CITIES = [  # (lon, lat), desc, horizontal alignment
    [(141.0281, 37.4213), '  Daiichi Nuclear Power Plant', 'left'],
    [(141.0125, 37.4492), 'Futaba ', 'right'],
    [(141.0000, 37.4833), ' Namie', 'left'],
    [(140.9836, 37.4044), ' Okuma ', 'right'],
    [(141.0088, 37.3454), ' Tomioka', 'left']]

# The coordinates our analysis will cover
X_COORDS = (140.9, 141.3)
Y_COORDS = (37.0, 37.83)

# The city coordinates need to be normalized to our grid (whose size depends on CELLS_PER_SIDE)
norm_x = [CELLS_PER_SIDE*((c[0][0]-X_COORDS[0])/(X_COORDS[1]-X_COORDS[0])) for c in CITIES]
norm_y = [CELLS_PER_SIDE*((c[0][1]-Y_COORDS[0])/(Y_COORDS[1]-Y_COORDS[0])) for c in CITIES]

# Create the data access object factory.
factory = sina_sql.DAOFactory(DATABASE)
record_handler = factory.createRecordDAO()
relationship_handler = factory.createRelationshipDAO()

# Get the ids of the experiments (which are their dates)
all_experiments = record_handler.get_all_of_type("exp")
dates = [str(x.id) for x in all_experiments]
print('Database has the following dates available: {}'.format(', '.join(dates)))

Filter the Data: Filtering Logic
========================
We subdivide our coordinate range (37.3-37.8, 140.9-141.4) into $cells\_per\_side^2$ regions and find the records whose coordinates are within each range. We separate these out based on which day each Record is associated with. We then find that Record's gcnorm (counts per sec) and average to get that cell's average for the day, and also track the total number of records per cell per day (so we know around how confident we are in that average). 

This cell adds the functions to memory, plus does a bit of preprocessing. The functions themselves will be called once it's time to create the graph.

In [ ]:
from sina.utils import ScalarRange
from collections import defaultdict
import random
import matplotlib.pyplot as plt

# First, we figure out which record ids are associated with which dates
records_at_dates = {}
for date in dates:
    records_at_dates[date] = set([str(x.object_id) for x in relationship_handler.get(subject_id=date,
                                                                                     predicate="contains")])
    
# Jupyter sometimes has an issue with the first call to plt.show(), so we make a dummy call
plt.show()


def find_in_range(lat_min, lat_max, long_min, long_max):
    """
    Find and return all Records in a coordinate square.
    
    :param lat_min: The minimum latitude a Record can have (inclusive)
    :param lat_max: The maximum latitude a Record can have (exclusive)
    :param long_min: The minimum longitude a Record can have (inclusive)
    :param long_max: The maximum longitude a Record can have (exclusive)
    
    :returns: A list of Records within this coordinate range
    """
    latitude_req = ScalarRange(name="latitude",
                               min=lat_min,
                               min_inclusive=True,
                               max=lat_max,
                               max_inclusive=False)
    longitude_req = ScalarRange(name="longitude",
                                min=long_min,
                                min_inclusive=True,
                                max=long_max,
                                max_inclusive=False)
    return record_handler.get_given_scalars((latitude_req,
                                             longitude_req))
    
    
def calculate_cell(lat_min, lat_max, long_min, long_max):
    """
    Calculate the avg_gcnorm and count for the cell across each day in records_at_dates.
    
    :param lat_min: The minimum latitude of this cell (inclusive)
    :param lat_max: The maximum latitude of this cell (exclusive)
    :param long_min: The minimum longitude of this cell (inclusive)
    :param long_max: The maximum longitude of this cell (exclusive)
    
    :returns: a dictionary mapping total and average gcnorm & num samples in a cell to a day
    """
    records = find_in_range(lat_min, lat_max, long_min, long_max)
    out = defaultdict(lambda: {"total":0.0, "count":0, "average":0.0})
    for record in records:
        for date in records_at_dates:
            if record.id in records_at_dates[date]:
                out[date]["total"] += record.data["gcnorm"]["value"]
                out[date]["count"] += 1
                break
    for date in records_at_dates:
        if out[date]["count"] > 0:
            out[date]["average"] = out[date]["total"]/(out[date]["count"])
    return out

print("Functions loaded and date mappings built!")

Create the Graphs
===============
Now we divide up based on the number of cells and collect the information for each cell independently. Since we're only *reading* the underlying database, this could, in theory, be parallelized. Generating this graph may take some time; see the progress indicator beneath the code for an idea of how much is left.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import patheffects
import numpy as np
from IPython.display import clear_output, display

x_increment = (X_COORDS[1]-X_COORDS[0])/CELLS_PER_SIDE
x_range = [x_increment*offset+X_COORDS[0] for offset in range(CELLS_PER_SIDE)]
y_increment = (Y_COORDS[1]-Y_COORDS[0])/CELLS_PER_SIDE
y_range = [y_increment*offset+Y_COORDS[0] for offset in range(CELLS_PER_SIDE)]

avgs_at_date = defaultdict(lambda: np.zeros((CELLS_PER_SIDE, CELLS_PER_SIDE)))
counts_at_date = defaultdict(lambda: np.zeros((CELLS_PER_SIDE, CELLS_PER_SIDE)))

# This may take awhile! (around a minute for a 12*12 map)
def gen_plot():  
    """Generate the plot, including calculating the data it contains."""
    cells_completed = 0
    for x_offset, x_coord in enumerate(x_range):
        for y_offset, y_coord in enumerate(y_range):
            norms_at_dates = calculate_cell(lat_min=y_coord,
                                             lat_max=y_coord + y_increment,
                                             long_min=x_coord,
                                             long_max=x_coord + x_increment)
            cells_completed += 1
            progress = ("Progress: {}/{}, finished ([{},{}), [{}, {}))"
                        .format(cells_completed,
                         CELLS_PER_SIDE**2,
                         '%.3f'%(x_coord),
                         '%.3f'%(x_coord + x_increment),
                         '%.3f'%(y_coord),
                         '%.3f'%(y_coord + y_increment)))
            clear_output(wait=True)
            display(progress)
            for date in norms_at_dates:
                avgs_at_date[date][y_offset, x_offset] = (norms_at_dates[date]["average"])
                counts_at_date[date][y_offset, x_offset] = (norms_at_dates[date]["count"])
    create_graph()
                
                
def create_graph():
    """
    Configure and display the graph itself. Dependent on the data from gen_plot().
    
    In case you're trying different graph display configs, you may want to call this
    instead of gen_plot(); as long as gen_plot()'s been called once in this notebook
    instance (and the kernel hasn't restarted), the data it needs will still be in
    memory, and calling this alone is much faster.
    """
    print("Creating graph...")
    for date in dates:
        fig, ax = plt.subplots(figsize=(9,9))
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        heatmap_avg = ax.imshow(avgs_at_date[date], origin='lower', cmap=COLORMAP)
        plt.colorbar(heatmap_avg, label="Counts Per Second")
        plt.title("Fukushima Radiation: Flight {}".format(date))
        scatter = ax.scatter(x=norm_x,
                             y=norm_y,
                             s=AREA_CITIES,
                             c=COLOR_OUTLINE)
        scatter = ax.scatter(x=norm_x,
                             y=norm_y,
                             s=(AREA_CITIES/2.0),
                             c=COLOR_CITIES)
        for x_coord, y_coord, city_info in zip(norm_x, norm_y, CITIES):
            _, desc, alignment = city_info
            text = ax.text(x_coord, y_coord,desc,
                           va="center", ha=alignment,
                           size=FONT_SIZE, color=COLOR_CITIES)
            text.set_path_effects([patheffects.withStroke(linewidth=OUTLINE_SIZE,
                                                          foreground=COLOR_OUTLINE)])

        # Matplotlib labels the boxes themselves, rather than their
        # borders/the origins, so we need to calculate the centers
        ax.set_xticks(range(len(x_range)))
        ax.set_xticklabels(('%.3f'%(x+x_increment/2) for x in x_range))
        ax.set_yticks(range(len(y_range)))
        ax.set_yticklabels(('%.3f'%(y+y_increment/2) for y in y_range))
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
                 rotation_mode="anchor")
        
        plt.show()

gen_plot()